# The KMALL Data Reader
This KMALL data reader is incomplete, but a good start. The following will guide one through what's currently possible. At the end is a list of obvious things that are yet to be completed or might be in place, but warrant rethinking. 

In [1]:
import kmall

Each kmall object is associated with a datafile. So when creating an object, pass the filename to be associated with it.

In [4]:
filename = "/Users/vschmidt/scratch/20190429/sisdata/raw/20190426_3/0000_20190426_185811_ShipName.kmall"
K = kmall.kmall(filename)


The file can be indexed with the following. This happens automatically when reading the file for other purposes, but sometimes you want the index itself, so you can call it directly as shown here. The index itself is a pandas DataFrame, and we can look at first several rows of the index to get an idea of what's inside. 

In [5]:
K.index_file()
print(K.Index.iloc[0:20,:])

              ByteOffset  MessageSize MessageType
Time                                             
1.556305e+09           0         1054     b'#IIP'
1.556305e+09        1054         1328     b'#IOP'
1.556305e+09        2382          332     b'#SVP'
1.556305e+09        2714           68     b'#SVT'
1.556305e+09        2782           68     b'#SVT'
1.556305e+09        2850           68     b'#SVT'
1.556305e+09        2918           68     b'#SVT'
1.556305e+09        2986           68     b'#SVT'
1.556305e+09        3054           68     b'#SVT'
1.556305e+09        3122           68     b'#SVT'
1.556305e+09        3190           68     b'#SVT'
1.556305e+09        3258           68     b'#SVT'
1.556305e+09        3326           68     b'#SVT'
1.556305e+09        3394           68     b'#SVT'
1.556305e+09        3462          156     b'#SPO'
1.556305e+09        3618        50028     b'#MRZ'
1.556305e+09       53646        49930     b'#MRZ'
1.556305e+09      103576        50112     b'#MRZ'


It is often useful to have a summary of the types of packets in a file. This can be done with the `report_packet_types()` method.

In [6]:
K.report_packet_types()

             Count      Size:  Min Size  Max Size
MessageType                                      
b'#CPO'         63       9786       154       156
b'#IIP'          1       1054      1054      1054
b'#IOP'          1       1328      1328      1328
b'#MRZ'       3578  178847332     49028     51140
b'#SCL'         63       4788        76        76
b'#SKM'         62     835548     13368     13500
b'#SPO'         63       9786       154       156
b'#SVP'          1        332       332       332
b'#SVT'       1579     107372        68        68


When Kongsberg system installation suffer from networking problems (or hardware malfunctions of the CPU), they sometimes loose data packets.  While there is indication in SIS 4/5 if incoming data to the PPU (navigation, attitude, velocity) has gaps, there is often not any indication that the sonar records coming out of the PPU have gaps. If the extent of the problem is great enough, there will be a "Failure to report depths." error, but if not, the system will silently log data with the occasional missing record. 

In an effort to try to detect this in logged data files a routine has been written, `check_ping_count()`, which compares the ping indices (these values increase with each ping cycle, but repeat for each swath within a ping cycle), indices of expected receive fans and tne number of received *MRZ* records (there should be 1 per receive fan), and reports anything missing. 

In the future, the check will also audit the navigation and attitude data (which are stored in the same record when supplied by a POS/MV Group record via Ethernet) and report gaps in the record when they exist. This is done currently in if the module is run as a script on a file (more about that later), but not yet implemented as a class method.

Here's an example, where the results are both printed out by default and returned as a tuple for internal use later.

In [7]:
result = K.check_ping_count()

                                              File  NpingsTotal  Pings Missed  MissingMRZRecords
 /Users/vschmidt/scratch/20190429/sisdata/raw/2...         1790             1                  0


In this example, there should be 238 pings based on the difference in first and last ping indices, but two pings were missed in the middle.  However all the MRZ records associated with each existing ping record were found. 

In the future there will be utilty functions to make this process easier, for now one must extract desired data manually. Not all records can be read yet, but reading of complete MRZ records is supported. First lets filter the index for MRZ records: 

In [8]:
iMRZ = K.Index["MessageType"] == "b'#MRZ'"
MRZIndex = K.Index[iMRZ]
MRZIndex.head()

,ByteOffset,MessageSize,MessageType
Time,,,
1.556305e+09,3618,50028,b'#MRZ'
1.556305e+09,53646,49930,b'#MRZ'
1.556305e+09,103576,50112,b'#MRZ'
1.556305e+09,153688,50078,b'#MRZ'
1.556305e+09,203766,49948,b'#MRZ'


Now we can open the file, seek to the first record location and read the record. 

In [12]:
K.OpenFiletoRead()
K.FID.seek(MRZIndex["ByteOffset"].iloc[0],0)
dg = K.read_EMdgmMRZ()
print("MRZ Records:  " + ",".join( dg.keys()))
print("Soundings Record Fields: " + ",\n\t".join(dg["sounding"].keys()))

MRZ Records:  header,partition,cmnPart,pingInfo,txSectorInfo,rxInfo,extraDetClassInfo,sounding,SIsample_desidB
Soundings Record Fields: soundingIndex,
	txSectorNumb,
	detectionType,
	detectionMethod,
	rejectionInfo1,
	rejectionInfo2,
	postProcessingInfo,
	detectionClass,
	detectionConfidenceLevel,
	padding,
	rangeFactor,
	qualityFactor,
	detectionUncertaintyVer_m,
	detectionUncertaintyHor_m,
	detectionWindowLength_sec,
	echoLength_sec,
	WCBeamNumb,
	WCrange_samples,
	WCNomBeamAngleAcross_deg,
	meanAbsCoeff_dbPerkm,
	reflectivity1_dB,
	reflectivity2_dB,
	receiverSensitivityApplied_dB,
	sourceLevelApplied_dB,
	BScalibration_dB,
	TVG_dB,
	beamAngleReRx_deg,
	beamAngleCorrection_deg,
	twoWayTravelTime_sec,
	twoWayTravelTimeCorrection_sec,
	deltaLatitude_deg,
	deltaLongitude_deg,
	z_reRefPoint_m,
	y_reRefPoint_m,
	x_reRefPoint_m,
	beamIncAngleAdj_deg,
	realTimeCleanInfo,
	SIstartRange_samples,
	SIcentreSample,
	SInumSamples


In [11]:
dg

{'header': {'numBytesDgm': 50028,
  'dgmType': b'#MRZ',
  'dgmVersion': 0,
  'systemID': 40,
  'echoSounderID': 2040,
  'dgtime': 1556305090.446813,
  'dgdatetime': datetime.datetime(2019, 4, 26, 18, 58, 10, 446813)},
 'partition': {'numOfDgms': 1, 'dgmNum': 1},
 'cmnPart': {'numBytesCmnPart': 12,
  'pingCnt': 25095,
  'rxFansPerPing': 2,
  'rxFanIndex': 0,
  'swathsPerPing': 2,
  'swathAlongPosition': 0,
  'txTransducerInd': 0,
  'rxTransducerInd': 0,
  'numRxTransducers': 1,
  'algorithmType': 0},
 'pingInfo': {'numBytesInfoData': 144,
  'padding0': 0,
  'pingRate_Hz': 28.08705711364746,
  'beamSpacing': 2,
  'depthMode': 1,
  'subDepthMode': 0,
  'distanceBtwSwath': 0,
  'detectionMode': 0,
  'pulseForm': 0,
  'padding1': 0,
  'frequencyMode_Hz': 300000.0,
  'freqRangeLowLim_Hz': 260000.0,
  'freqRangeHighLim_Hz': 320000.0,
  'maxTotalTxPulseLength_sec': 0.00010099999781232327,
  'maxEffTxPulseLength_sec': 3.7369998608483e-05,
  'maxEffTxBandWidth_Hz': 14283.0,
  'absCoeff_dBPerkm':

There is also a debugging method `print_datagram()` for printing the fields of a record. It is very verbose, but can be helpful to dump everything to sort out a problem. Here's an example on the MRZ header, which is not so large.

In [10]:
K.print_datagram(dg["header"])



numBytesDgm:			50028

dgmType:			b'#MRZ'

dgmVersion:			0

systemID:			40

echoSounderID:			2040

dgtime:			1556305090.446813

dgdatetime:			2019-04-26 18:58:10.446813



## What's next:
Here's a list of obvious additions and improvements to the reader:

1. The installation parameters datagram can be read, but the text string cannot yet be parsed.
2. The runtime parameters datagram can be read, but the text string cannot yet be parsed. 
3. The file Index is indexed by time in Unix format. These could/should be converted to human readable times.
4. In file index messag type is not a simple "MRZ" but rather the text "b'#MRZ'". This could be simplified.
5. There is not yet a read_next_datagram() method, which can be useful to walk through a file. (although the index helps)
6. There is not yet a utilty function that can extract all the sounding data in x,y,z re vessel and x,y,z in geographic coordinates and meters for a) the ping and b) all pings between two indices and c) the whole file. 
7. The packets related to BIS error reports, reply, and short reply cannot yet be read / interpreted. 
8. The water column datagram, #MWC, cannot yet be read.